# Porosimetry Analysis

## Import Libraries

In [1]:
#importing custom functions
import sys
sys.path.append('/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/python_analysis/common_functions')
from custom_functions import *
from common_fits import *

import os
import io
import codecs
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import iqr
import matplotlib.pyplot as plt
import seaborn as sns
import time
from statannot import add_stat_annotation
import itertools
import shutil
from scipy.optimize import curve_fit

# Constants

## Directories

In [2]:
#Initialisation of data read
#Root location
root = '/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/'
#current location
location = os.getcwd()
print(location)

#Data set locations
#Hollow fibre micro-fibre analysis
#Effect of pyridine concentration with S4 polymer solution on HF membrane porosity
hf_s4_pyrid = root+'transport/porosimetery/raw_data/hollow_fibre/txt/'
#Effect of pyridine concentration with S4 polymer solution on flatsheet membrane porosity
fs_s4_pyrid = root+'transport/porosimetery/raw_data/flatsheet/txt/'
#Roxanna's Data
R_dat_loc = root+'transport/porosimetery/Roxanna/'

#Penetrometer blank data
blank_loc = root+'transport/porosimetery/blanks/txt/'

#pressure table
p_table = np.loadtxt(root+'transport/porosimetery/raw_data/pressure_table.txt', delimiter="\t")
#/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/transport/porosimetery/raw_data

#List of all data sets to be analysed
data_sets = [hf_s4_pyrid,fs_s4_pyrid]

#sample key location
sample_key_name = 'sample_key_17032020'
sample_key = '/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/sample_keys/'+sample_key_name+'.xlsx'

#Import sample key
sample_key = pd.read_excel(sample_key)

#Set location of processed data
processed = location+'/processed/'
checkdir(processed) #check that processed data directory exists
#location of processed collated porosimetry data
collated_data= processed +'/'+'collated_data'+'/'
checkdir(collated_data) #check that collated data directory exists
#location of processed and binned raw distribtions
raw_dists = processed+'raw_dist/'
checkdir(raw_dists) #check that processed recreated raw distribtion data directory exists
#location of the blanked data
blank_sav = processed +'blanked/'
checkdir(blank_sav) #check that blanked data save location exists

/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/python_analysis/porosimetry


## Variable Parameters

In [3]:
#set variables to be considered to separate out distributions
variables = ['pyridine_conc','flat_or_fibre']
#create a dictionary of variable titles for each of the variables considered
variable_label = {'solution_name':'Polymer Solution','pyridine_conc':'Pyridine Concentration (PPM)','wire_speed':'Wire Speed (mm/s)',
                  'rotation_speed':'Rotation Speed (degrees/s)','flow_rate':'Flow Rate (ml/hr)','flat_or_fibre':'Membrane Morphology'} 

## Constant parameters

In [4]:
#Penetrometer parameters
Penetrometers = {'14-0783':{'Penetrometer':'3 Bulb, 06-11, Stem, Solid','stem_vol':0.4120, 'pen_vol':3.2121, 'pen_weight':62.4435},
                '14-0638':{'Penetrometer':'3 Bulb, 06-11, Stem, Solid','stem_vol':0.4120, 'pen_vol':3.2121, 'pen_weight':62.6965}}

#Hg parameters
hg_contact_angle = 130 * (np.pi/180) #degrees
hg_surface_tension = 485 #dynes/cm
hg_density = 13.5335 #g/ml

#porosity data processing constants
#Washburn constant
WASHCON = (10**4)/68947.6 #((um/cm)/(dynes/((cm**2)*psia)))

#Accepted R^2 value
accept_r2 = 0.9

## Import Data

### Common import function from TXT file

In [5]:
def TXT_import(data_loc,data,penetrometer):
    ##Opening file    
    #select file,open it and create handle for the file
    fhand = codecs.open(os.path.join(data_loc,data),'r', 'iso-8859-1')
    
    #Extracting intrustion data
    #to find data within txt file start counters to specify line within txt file
    count = 0
    #create place holders for where want to slice txt file
    start = 0
    end = 0
    #create string of all data in file
    ex_data = []
    #for each line in the file
    for line in fhand:
        #add one to counter to keep track of which file line
        count = count + 1
        #remove (strip) white space
        line = line.strip()
        #add the each line to the string of all data
        ex_data.append(line)
        #find line in file which starts with tabular, as this is beginning of raw p-v data in file
        if 'Cumulative Intrusion vs Pressure' in line:
            start = count
        #find line in file 'Cumulative Intrusion vs Pressure', as this is beginning of next section after raw p-v data in file
        if 'Incremental Intrusion vs Pressure' in line:
            end = count
    #slice string with data between beginning and end of raw data to extract only data
    ex_data = ex_data[start:end]
    #print(data)
    
    #Having removed data bounds from bulk file, will now trim to extract only numerical data
    #to find data within txt file start counter to specify line within txt file
    count = 0
    #create place holders for where want to slice txt file
    start = 0
    end = 0
    #for each line withing the data
    for line in ex_data:
        #add one to counter to keep track of which file line
        count = count + 1
        #find line in file which starts with ----:, as this is beginning of raw p-v numerical data in file
        if line.startswith('Pressure (psia)'):
            start = count
        #take note of point just before MICROMETRICS as this is where numerical data ends within trimmed data file    
        if line.startswith('MICROMERITICS'):
            end = count - 4
    #slice string with data between beginning and end of raw data to extract only numerical data
    ex_data = ex_data[start:end]
    #print(ex_data)

    #Having extracted string of all numerical data need to convert into pandas dataframe
    pddata = pd.read_csv(io.StringIO('\n'.join(ex_data)),header=None, delim_whitespace=True)
    #With the correct column names
    pddata.columns = ['pressure', 'cumulative_intrusion']    
    #print(pddata.head())
    
    #Bin cumulative_intrusions to standard pressure table to allow for constant pressure table to be used and thereby allow for averaging data
    bin_array = p_table
    #bin data according to pressure table
    bin_sum, bin_edges, binnumber = stats.binned_statistic(pddata['pressure'], pddata['cumulative_intrusion'], statistic ='sum', bins = bin_array)
    #finding the middle of each bin 
    bin_width = (bin_edges[1] - bin_edges[0])
    bin_centers = bin_edges[1:] - bin_width/2
    #creating binned dataframe with sum of each bin as column
    pddata = pd.DataFrame({'pressure':bin_centers, 'cumulative_intrusion':bin_sum})
    #print(pddata)
    #replace 0 values with previous value in table
    #make list for values
    P_dat = []
    for i in range(len(pddata['cumulative_intrusion'])):
        if i == 0:
            P_dat.append(pddata['cumulative_intrusion'].iloc[i])
        elif pddata['cumulative_intrusion'].iloc[i] == 0:
            P_dat.append(P_dat[i - 1])
        #elif P_dat[i-1] > pddata['cumulative_intrusion'].iloc[i]:
        #    P_dat.append(P_dat[i - 1])
        else:
            P_dat.append(pddata['cumulative_intrusion'].iloc[i])
            pass
    #print(P_dat)
    pddata['cumulative_intrusion'] = P_dat
    
    #Extract total volume of mercury injected into penetrometer
    V_tot = P_dat[-1] 
    
    ##Extracting sample weight
    #select file,open it and create handle for the file
    fhand = codecs.open(os.path.join(data_loc,data),'r', 'iso-8859-1')
    #to find data within txt file start counters to specify line within txt file
    count = 0
    #create place holders for where want to slice txt file
    start = 0
    end = 0
    #create string of all data in file
    ex_data = []
    #for each line in the file
    for line in fhand:
        #remove (strip) white space
        line = line.strip()
        #add the each line to the string of all data
        ex_data.append(line)
        #find line in file which starts with tabular, as this is beginning of raw p-v data in file
        if 'Sample Weight' in line:
            start = count
            end = count + 1
        #add one to counter to keep track of which file line
        count = count + 1
    #slice string with data between beginning and end of raw data to extract only data
    ex_data = ex_data[start:end]
    sample_weight = float(ex_data[0][-8:-2])
    
    ##Extracting assembly weight
    #select file,open it and create handle for the file
    fhand = codecs.open(os.path.join(data_loc,data),'r', 'iso-8859-1')
    #to find data within txt file start counters to specify line within txt file
    count = 0
    #create place holders for where want to slice txt file
    start = 0
    end = 0
    #create string of all data in file
    ex_data = []
    #for each line in the file
    for line in fhand:
        #remove (strip) white space
        line = line.strip()
        #add the each line to the string of all data
        ex_data.append(line)
        #find line in file which starts with tabular, as this is beginning of raw p-v data in file
        if 'Assembly Weight:' in line:
            start = count
            end = count + 1
        #add one to counter to keep track of which file line
        count = count + 1
    #slice string with data between beginning and end of raw data to extract only data
    ex_data = ex_data[start:end]
    #print(ex_data)
    assembly_weight = float(ex_data[0][-13:-3])
    #print(assembly_weight)
    
    ###Begin processing extracted data
    ##Calculating pore size distribution
    #convert pressures into characteristic pore diameters
    #get conversion equation from micromeritics data processing
    #D = (WASHCON*y*(-4cos(q)))/P
    #Calculate diameters from associated pressure and then convert diameters from micrometers into nanometers
    pddata['pore_diameter'] = ((WASHCON*hg_surface_tension*(-4*np.cos(hg_contact_angle)))/pddata['pressure'])* 10**3
    
    #calculate incremental specific intrusion volume
    #from micromeritics data processing
    #Iii = Ii - Ii-1 
    Ii = []
    counter = 0
    for intrusion in pddata['cumulative_intrusion']:
        if counter == 0:
            Iii = 0
        else:
            Iii = pddata.iloc[counter]['cumulative_intrusion'] - pddata.iloc[counter-1]['cumulative_intrusion']
        Ii.append(Iii)
        counter = counter + 1

    #calculate differential specific incremental intrusion volume
    #from micromeritics data processing
    #Idi = Ii/(Di - (Di-1))
    dI = []
    counter = 0
    for intrusion in Ii:
        if counter == 0:
            dIi = (-Ii[counter])/(pddata['pore_diameter'].iloc[counter])
        else:
            dIi = (-Ii[counter])/(pddata['pore_diameter'].iloc[counter]-pddata['pore_diameter'].iloc[counter-1])
        dI.append(dIi)
        counter = counter + 1
    
    
    #calculate log differential diameter incremental intrusion volume
    #from micromeritics data processing
    #Idi = Ii/(log(Di) - log((Di-1)))
    log_dI = []
    counter = 0
    for intrusion in Ii:
        if counter == 0:
            log_dIi = (-Ii[counter])/(np.log10(pddata['pore_diameter'].iloc[counter]))
        else:
            log_dIi = (-Ii[counter])/(np.log10(pddata['pore_diameter'].iloc[counter])-np.log10(pddata['pore_diameter'].iloc[counter-1]))
        log_dI.append(log_dIi)
        counter = counter + 1
    
    #print(pddata)
    pddata = pd.concat([pddata,pd.DataFrame({'SIV':np.transpose(Ii),'DSIV':np.transpose(dI),'log_DSIV':np.transpose(log_dI)})], axis=1)
    #print(pddata)

    #retruning dataframe
    return pddata, assembly_weight,sample_weight,V_tot

## Import blanking data

In [6]:
for filename in os.listdir(blank_loc):
    if filename.endswith(".TXT") or filename.endswith(".txt"):
        print(filename)
        #identify penetrometer blank associated with
        penetrometer = str(sample_key.loc[sample_key['porosimetery_filename'] == filename, 'penetrometer'].iloc[0])
        #Open and process blank data file
        blank_df, assembly_weight,sample_weight,b_V_tot = TXT_import(blank_loc,filename,penetrometer)
        #Save processed pore distribution file to csv
        blank_df.to_csv(blank_sav+penetrometer+'.csv', index=False)
        #save blank pore information
        b_pore_info = [assembly_weight,sample_weight,b_V_tot]
        b_pore_info = pd.DataFrame((np.asarray(b_pore_info)))
        #print(blank_df.head())
        b_pore_info.to_csv(blank_sav+penetrometer+'.txt', index=False)

Blank_14-0783_06-11_15012020.TXT
Blank_14-0638_06-11_04022020.txt


## Import & processing of raw data

In [7]:
#Having processed blanking data file, will use this baseline to correct error in all imported data
#first import blank file so do not have to repeat for raw data file opened 
#blank_df = pd.read_csv(blank_sav+'blank_data.csv')
#print(blank_df.head())
for data_loc in data_sets:
    for filename in os.listdir(data_loc):
        if filename.endswith(".TXT") or filename.endswith(".txt"):
            #print(filename)
            #Intiially identify which penetrometer should be used
            penetrometer = str(sample_key.loc[sample_key['porosimetery_filename'] == filename, 'penetrometer'].iloc[0])
            #and open the associated penetrometer data file
            blank_df = pd.read_csv(blank_sav+penetrometer+'.csv')
            #import b_pore_info
            b_pore_info = pd.read_csv(blank_sav+penetrometer+'.txt',sep="\t",header=None)
            
            #Open dataset and extract poresize distribution, Hg_volume and skeletal_volume
            df,assembly_weight,sample_weight, V_tot = TXT_import(data_loc,filename,penetrometer)
            #print(df)
            #print(sample_weight)
            
            #to account for insturmental noise, want to subtract all blanked columns from sample data except for associated pore sizes
            #use difference to find columns which differ
            cols = df.columns.difference(['pressure','pore_diameter'])
            #then subtract columns from blank file from raw data
            df[cols] = df[cols].sub(blank_df[cols])
            #re-sort the dataframe according to it index
            df = df.reset_index(drop=True)
            #print(df)
            
            ## Processing porosity data
            ##Calculating bulk of enveloping volume
            #Import penetrometer info
            pen_weight = Penetrometers[penetrometer]['pen_weight']
            #Initially calculate the mass of mercury pressent in pentomieter at atmospheric pressure = Assembly mass - (Pentometer mass + sample mass)
            Hg_weight = assembly_weight - (pen_weight + sample_weight) # (g)
            #calculate volume of Hg from density
            Hg_volume = Hg_weight/hg_density # (ml)
            #Extract skeletal volume from final value of cumulative intrusion volume
            #V_tot = df['cumulative_intrusion'].iloc[-1] * sample_weight #(ml)
            V_tot = df['cumulative_intrusion'].iloc[-1] 
            #calculate bulk volume
            Vb = (Penetrometers[penetrometer]['pen_vol'] - Hg_volume)/sample_weight
            #calcluate porosity
            df.loc[0, 'porosity'] = (V_tot/Vb)*100
            #print((V_tot/Vb)*100)
            
            
            ##Adding in meta data associated with each curve
            for variable in variables:
                #save associated meta data to the first line of the dataframe
                df.loc[0, variable] = sample_key.loc[sample_key['porosimetery_filename'] == filename, variable].iloc[0]
            #add in unique identifier for easy additional metadata identification later on
            df.loc[0, 'unique_id'] = int(sample_key.loc[sample_key['porosimetery_filename'] == filename, 'unique_id'].iloc[0])
            #print(df.head())

            #Saving out binned dataset as a csv file so may be recalled later
            df.to_csv(raw_dists+str(filename[:-4])+'.csv')

### Import and processing of Roxanna's Data

In [8]:
#make data frame to hold all of roxanna's data in
Rox_dat = pd.DataFrame()
for filename in os.listdir(R_dat_loc):
    if filename.endswith(".TXT") or filename.endswith(".txt"):
        #print(filename)
        #identify penetrometer blank associated with
        penetrometer = str(sample_key.loc[sample_key['porosimetery_filename'] == filename, 'penetrometer'].iloc[0])
        #Open and process blank data file
        df = pd.read_csv(os.path.join(R_dat_loc,filename), sep="\t",header=None)
        #print(df.head())
        #extract pressure and cumulitive intrusion data
        pddata = pd.DataFrame({'pressure':df[0].iloc[1:],'cumulative_intrusion':df[1].iloc[1:]})
        #As headers are not taken, entire column initially strings. Convert to floats for binning
        pddata = pddata.apply(pd.to_numeric, errors='coerce')
        
        #Bin cumulative_intrusions to standard pressure table to allow for constant pressure table to be used and thereby allow for averaging data
        bin_array = p_table
        #bin data according to pressure table
        bin_sum, bin_edges, binnumber = stats.binned_statistic(pddata['pressure'], pddata['cumulative_intrusion'], statistic ='sum', bins = bin_array)
        #finding the middle of each bin 
        bin_width = (bin_edges[1] - bin_edges[0])
        bin_centers = bin_edges[1:] - bin_width/2
        #creating binned dataframe with sum of each bin as column
        pddata = pd.DataFrame({'pressure':bin_centers, 'cumulative_intrusion':bin_sum})
        #print(pddata)
        #replace 0 values with previous value in table
        #make list for values
        P_dat = []
        for i in range(len(pddata['cumulative_intrusion'])):
            if i == 0:
                P_dat.append(pddata['cumulative_intrusion'].iloc[i])
            elif pddata['cumulative_intrusion'].iloc[i] == 0:
                P_dat.append(P_dat[i - 1])
            else:
                P_dat.append(pddata['cumulative_intrusion'].iloc[i])
                pass
        #save the cumulative intrusion as that, with zeros replaced
        pddata['cumulative_intrusion'] = P_dat
        #print(pddata['cumulative_intrusion'])
        
        #Import sample weight
        sample_weight = float(df[4].iloc[0])
        #Import assembly weight
        assembly_weight = float(df[4].iloc[1])
        
        #calculate porosity
        #Import penetrometer info
        pen_weight = Penetrometers[penetrometer]['pen_weight']
        #Initially calculate the mass of mercury pressent in pentomieter at atmospheric pressure = Assembly mass - (Pentometer mass + sample mass)
        Hg_weight = assembly_weight - (pen_weight + sample_weight) # (g)
        #calculate volume of Hg from density
        Hg_volume = Hg_weight/hg_density # (ml)
        #Extract skeletal volume from final value of cumulative intrusion volume
        #V_tot = df['cumulative_intrusion'].iloc[-1] * sample_weight #(ml)
        V_tot = pddata['cumulative_intrusion'].iloc[-1] 
        #skeletal_volume = pddata['cumulative_intrusion'].iloc[-1]

        #secondary method for porosimetry
        Vb = (Penetrometers[penetrometer]['pen_vol'] - Hg_volume)/sample_weight
        #calcluate porosity
        #df.loc[0, 'porosity'] = (V_tot/Vb)*100
        porosity = (V_tot/Vb)*100
        
        #add to Roxanna's dataframe
        Rox_dat.loc[filename,'porosity'] = porosity

#Sort Roxanna dataframe from highest to lowest
#Rox_dat = Rox_dat.sort_values(columns='porosity',axis=1)
Rox_dat.sort_values(by=['porosity'], inplace=True)
#print(Rox_dat)
#save roxanna's data frame out
Rox_dat.to_csv(processed+'/rox_dat/'+'rox_dat.csv')

# Separation of distributions by variables

## Initially create a dictionary of all files which have the same parameters used

In [9]:
para_comb = SimilarSetCollation(raw_dists,variables,sample_key,'porosimetery_filename','txt')
print(para_comb)

{'25.0,0.0': ['S4_25PPM_POLYU_2.csv', 'S4_25PPM_POLYU_3.csv', 'S4_25PPM_POLYU_1.csv'], '25.0,1.0': ['S4_25PPM_F_2.csv', 'S4_25PPM_F_3.csv', 'S4_25PPM_F_1.csv'], '5.0,1.0': ['S4_5PPM_F_3.csv', 'S4_5PPM_F_2.csv', 'S4_5PPM_F_1.csv'], '0.0,1.0': ['S4_0PPM_F_7.csv', 'S4_0PPM_F_9.csv', 'S4_0PPM_F_8.csv'], '50.0,0.0': ['S4_50PPM_POLYU_6.csv', 'S4_50PPM_POLYU_5.csv', 'S4_50PPM_POLYU_4.csv'], '10.0,1.0': ['S4_10PPM_F_3.csv', 'S4_10PPM_F_2.csv', 'S4_10PPM_F_1.csv'], '0.0,0.0': ['S4_0PPM_POLYU_3.csv', 'S4_0PPM_POLYU_2.csv', 'S4_0PPM_POLYU_1.csv'], '1.0,0.0': ['S4_1PPM_POLYU_1.csv', 'S4_1PPM_POLYU_2.csv', 'S4_1PPM_POLYU_3.csv'], '1.0,1.0': ['S4_1PPM_F_1.csv', 'S4_1PPM_F_3.csv', 'S4_1PPM_F_2.csv'], '10.0,0.0': ['S4_10PPM_POLYU_3.csv', 'S4_10PPM_POLYU_2.csv', 'S4_10PPM_POLYU_1.csv'], '50.0,1.0': ['S4_50PPM_F_2.csv', 'S4_50PPM_F_3.csv', 'S4_50PPM_F_1.csv'], '5.0,0.0': ['S4_5PPM_POLYU_3.csv', 'S4_5PPM_POLYU_2.csv', 'S4_5PPM_POLYU_1.csv']}


## Combination of files with the same parameters

In [10]:
#Initially itterating through each of the different filename associated with each of the parameter combination identifiers
for identifier in para_comb:
    #initilise counter to keep track of how many files are being considered for each parameter set
    file_count = 0
    #As want to initially keep extracted intrusion data separate from pressure table create holding dataframe
    ex_df = pd.DataFrame()
    #At the same time want to extract porosity data, create list of extracted porosity data
    ex_poro = []
    #make list of unique ideitifiers of averaged datasets
    uni_ids = []
    for filename in para_comb[identifier]:
        #print(filename)
        #Initially want to open each file but check that is csv file
        if filename.endswith(".csv"):
            #open data file
            df = pd.read_csv(raw_dists+filename, index_col = 0)
            #print(df.head())
            #To begin averaging data between data sets extract pressure and cumulative intrusion for each
            #as only want to extract pore diameter table once only extract if considering first file
            if file_count == 0:
                para_df = df['pore_diameter'].copy()          
            #having extracted pressure table want to extract log differential diameter incremental intrusion volume associated with each
            ex_df = pd.concat([ex_df, df['log_DSIV']], axis=1,sort=False,ignore_index=True)
            # At the same time extract 
            ex_poro.append(df.loc[0,'porosity'])
            #extract unique identifiers
            uni_ids.append(df.loc[0,'unique_id'])
            #add to file_count to progress
            file_count = file_count + 1
    #having extracted the data from all datasets with the same parameter values want to calculate the average of each reading and associate with pressure again
    para_df = pd.concat([para_df, ex_df.mean(axis=1)], axis=1,sort=False,ignore_index=True)
    #print(para_df.head())
    #rename column headers
    para_df = para_df.rename(columns={0: "pore_diameter", 1: "avg_log_DSIV"})
    #print(para_df.head())
    
    #append unique identifiers to df for ease of metadata identification
    para_df = pd.concat([para_df, pd.DataFrame({'unique_id':uni_ids})], axis=1,sort=False)
    #print(para_df.head())
    #re-sort the dataframe according to it index
    para_df = para_df.sort_index(axis=0)
    ##Add associated meta_data to DataFrame and at the same time generate save name to reflect parameters used
    #make list holder for save name
    save_name = []
    for variable in variables:
        #add each variable to data frame in the first row
        variable_val = sample_key.loc[sample_key['unique_id'] == df.loc[0,'unique_id'], variable].iloc[0]
        para_df.loc[0, variable] = variable_val
        #append to save name
        save_name.append(variable+'-'+str(variable_val))
        #print(df.head())
    #strip save name
    save_name = str(save_name).strip('[]').replace("'",'').replace(' ','') 
    
    #append porosity data to log differential intrusion volume - pressure DataFrame
    para_df = pd.concat([para_df,pd.DataFrame(ex_poro,columns=['porosity'])], axis=1)
    #print(para_df.head())
    
    ##Saving averaged data set para_df out
    para_df.to_csv(collated_data+save_name+'.csv')

# Separation of distributions by variables

In [11]:
matched_para_comb = VariableSep(collated_data,variables)
print(matched_para_comb)

{'pyridine_conc, 5.0': {'flat_or_fibre': ['pyridine_conc-5.0,flat_or_fibre-1.0.csv', 'pyridine_conc-5.0,flat_or_fibre-0.0.csv']}, 'flat_or_fibre, 1.0': {'pyridine_conc': ['pyridine_conc-5.0,flat_or_fibre-1.0.csv', 'pyridine_conc-1.0,flat_or_fibre-1.0.csv', 'pyridine_conc-0.0,flat_or_fibre-1.0.csv', 'pyridine_conc-25.0,flat_or_fibre-1.0.csv', 'pyridine_conc-10.0,flat_or_fibre-1.0.csv', 'pyridine_conc-50.0,flat_or_fibre-1.0.csv']}, 'pyridine_conc, 1.0': {'flat_or_fibre': ['pyridine_conc-1.0,flat_or_fibre-1.0.csv', 'pyridine_conc-1.0,flat_or_fibre-0.0.csv']}, 'flat_or_fibre, 0.0': {'pyridine_conc': ['pyridine_conc-5.0,flat_or_fibre-0.0.csv', 'pyridine_conc-1.0,flat_or_fibre-0.0.csv', 'pyridine_conc-25.0,flat_or_fibre-0.0.csv', 'pyridine_conc-10.0,flat_or_fibre-0.0.csv', 'pyridine_conc-50.0,flat_or_fibre-0.0.csv', 'pyridine_conc-0.0,flat_or_fibre-0.0.csv']}, 'pyridine_conc, 0.0': {'flat_or_fibre': ['pyridine_conc-0.0,flat_or_fibre-1.0.csv', 'pyridine_conc-0.0,flat_or_fibre-0.0.csv']}, 'pyr

## Creating and deleting outputfiles

In [12]:
creation_and_destruction(processed,matched_para_comb)
print(matched_para_comb)

{'pyridine_conc, 5.0': {'flat_or_fibre': ['pyridine_conc-5.0,flat_or_fibre-1.0.csv', 'pyridine_conc-5.0,flat_or_fibre-0.0.csv']}, 'flat_or_fibre, 1.0': {'pyridine_conc': ['pyridine_conc-5.0,flat_or_fibre-1.0.csv', 'pyridine_conc-1.0,flat_or_fibre-1.0.csv', 'pyridine_conc-0.0,flat_or_fibre-1.0.csv', 'pyridine_conc-25.0,flat_or_fibre-1.0.csv', 'pyridine_conc-10.0,flat_or_fibre-1.0.csv', 'pyridine_conc-50.0,flat_or_fibre-1.0.csv']}, 'pyridine_conc, 1.0': {'flat_or_fibre': ['pyridine_conc-1.0,flat_or_fibre-1.0.csv', 'pyridine_conc-1.0,flat_or_fibre-0.0.csv']}, 'flat_or_fibre, 0.0': {'pyridine_conc': ['pyridine_conc-5.0,flat_or_fibre-0.0.csv', 'pyridine_conc-1.0,flat_or_fibre-0.0.csv', 'pyridine_conc-25.0,flat_or_fibre-0.0.csv', 'pyridine_conc-10.0,flat_or_fibre-0.0.csv', 'pyridine_conc-50.0,flat_or_fibre-0.0.csv', 'pyridine_conc-0.0,flat_or_fibre-0.0.csv']}, 'pyridine_conc, 0.0': {'flat_or_fibre': ['pyridine_conc-0.0,flat_or_fibre-1.0.csv', 'pyridine_conc-0.0,flat_or_fibre-0.0.csv']}, 'pyr

## Plotting

### Plotting pore size distribution

#### As Violin plot

In [36]:
#Routing through each level of the dictionary initally parasing each of matching parameter sets
for matching in matched_para_comb:
    #strip out whitespace from matching to find directories
    matchings = matching.replace(' ','') 
    #for each of the matching parameter sets route to the corresponding not matching parameters  
    for not_matching in matched_para_comb[matching]:
        #strip out whitespace from matching to find directories
        not_matchings = not_matching.replace(' ','') 
        #Make dataframe as holder for concatinated recreated data sets
        con_df = pd.DataFrame()
        #make dataframe for summary data
        sum_df = pd.DataFrame()
        #make list of not_matching variables for ordering
        nm_order = []
        #in each of these not matching parameters parase all of the listed files
        for filename in matched_para_comb[matching][not_matching]:
            #print(matching+not_matching)
            #print(filename)
            #create file to identify dataset
            file = filename[:-4]
            #print(file)
            #open distribution data as dataframe
            df = pd.read_csv(collated_data+filename, index_col = 0)
            #print(df.head())
            
            #extract not_matching parameter value
            #split between parameters
            nm = file.split(',')
            #split between parameter label and value
            nm = [i.split('-') for i in nm]
            #flatten list
            #nm = [item for sublist in nm for item in sublist]
            #print(nm)
            #exctract parameter value based on location within variables list
            #print(variables.index(not_matching))
            nm = nm[variables.index(not_matching)][1]
            #print(nm)
            
            #extend nm_order with not_matching parameter value
            nm_order.append(nm)
            
            #change data label to variable value
            df = df.rename(columns={'avg_log_DSIV':nm})
            
            #to allow for comparison of distributions want to look at normalised % frequency of each pore diameter
            #create a list of relative frequecies normalised to the total intrustion volume
            n_log_DSIV = ((df[nm].abs()/df[nm].abs().sum())*100).tolist()

            #create for loop that looks for any value less than 2 and removes, this also removes any nans
            for n, i in enumerate(n_log_DSIV):
                if i < 1.5:
                    n_log_DSIV[n] = float(0)
                elif str(i) == 'nan':
                    n_log_DSIV[n] = float(0)
            #print(n_log_DSIV)

            #create list of pore diamters
            pore_d = df['pore_diameter'].tolist()

            #create dictionary from list of frequencies and pore diameters
            d = {'n_log_DSIV':n_log_DSIV,'pore_diameter':pore_d}
            n_log_DSIV = pd.DataFrame(d)

            #create a list comprising a given radius the length of the total frequencies for each row
            rdf = sum([[row['pore_diameter']] * int(round(row['n_log_DSIV'])) for index, row in n_log_DSIV.iterrows()], [])
            #convert this list into a pandas dataframe
            rdf = pd.DataFrame({nm:rdf})
                    
            #Concatinate each recreated data set to eachother           
            con_df = pd.concat([con_df, rdf[nm]], axis=1, sort=False)
            #print(con_df)
            
            ##Calculating summary statistics from distriutions
            median = np.percentile(rdf[nm].dropna(), 50)
            IQR = iqr(rdf[nm].dropna())
            #in addition to the IQR calculate 25 and 75th quartiles
            quartile_25 = np.percentile(rdf[nm].dropna(), 25)
            quartile_75 = np.percentile(rdf[nm].dropna(), 75)
            mean = rdf[nm].mean()
            SD = rdf[nm].std()
            skew = rdf[nm].skew()
            #from calculation of skew determine whether to use median or mean
            if abs(skew) > 0.5:
                stat = 'median'
            else:
                stat = 'mean'

            #add each of the summary statstics and pair this with datasets
            sum_df.loc[nm,'median'] = median
            sum_df.loc[nm,'IQR'] = IQR
            sum_df.loc[nm,'25_quartile'] = quartile_25
            sum_df.loc[nm,'75_quartile'] = quartile_75
            sum_df.loc[nm,'mean'] = mean
            sum_df.loc[nm,'SD'] = SD
            sum_df.loc[nm,'skew'] = skew
            sum_df.loc[nm,'stat'] = stat
            

        #Check summary dataframe
        #print(sum_df.head())
        #save summary dataframe out as CSV
        sum_df.to_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_pore_dist_summary'+'.csv')
        
        #convert multiple columns of data into single column with header as variable
        data = pd.melt(con_df)

        #Find order of plots according to max value
        #ordered = data.groupby(['variable'])['value'].aggregate(np.median).reset_index().sort_values('value')
        #ordered_var = ordered['variable'].tolist()
        
        #Find order of plots from magnitude of nm variable, determined by ordering nm_order
        nm_order.sort(key = float, reverse=True)
        #print(nm_order)

        #create new template for figure
        fig, ax = plt.subplots()
        #plot violin plot into figure
        v_plt = sns.violinplot( x='variable', y= 'value', cut=0, data = data, order = nm_order, ax = ax) #order = ordered_var,
        
        #insert statistical annotations
        #before can add statistical annotation must create boxPairList from previous statistical comparison table
        #set which variable list controls order
        var_order = nm_order.copy()
        #create list for boxpairlist
        pre_boxPairList = []
        #for count of number of o values
        for index in range(len(var_order)):
            #to ensure that all combinations are considered again copy the uniquevalues
            avalues = var_order.copy() #colour hue
            #removing fixed variable so only consider changing variables
            avalues.remove(var_order[index])
            #considereing the appending value
            for index in range(len(avalues)):
                #let a = the file name and the ovalue which corresponds to the number within the list and pair them
                a = (avalues[index],var_order[index])
                #add the pair to the list of boxed pairs
                if avalues[index] != var_order[index]:
                    if a not in pre_boxPairList:
                        pre_boxPairList.append(a)
                else: pass
        #adding statistical annotation
        #if len(pre_boxPairList) > 1:
            #add_stat_annotation(x='variable', y= 'value', data = data, boxPairList=pre_boxPairList, 
            #                    test='Mann-Whitney', textFormat='star', loc='inside', verbose = 0,order = nm_order, ax = ax) #order = ordered_var,

        #Set plot labels
        #First add label to the x-axis to describe the variable considered
        #Retreve xlabel axis associated with variable parameter
        xlabel = variable_label[not_matching]

        #Add correct x tick labels
        #initially retreve the existing key labels
        labels = [t.get_text()  for t in ax.get_xticklabels()]

        xlabel_list = []
        for label in labels:
            #print(label)
            #For polymer solution crossreference variable with name
            if not_matching == 'solution_name':
                #initilise dictionary of polymer solution names
                polysolkey = {'0.0':'Trial','1.0':'Initial' ,'2.0':'S1','3.0':'S2','4.0':'S3','5.0':'S4','6.0':'S5'}
                #change label to update polymer solution name
                label = polysolkey[label]

            #append list to list of variable labels
            xlabel_list.append(label)
        #using list of variable labels update x-axis tick labels
        ax.set_xticklabels(xlabel_list)
        #set the x and y axis labels
        v_plt.set(xlabel=xlabel, ylabel='Pore Diameter (nm)') #
        #set the xlabels rotation
        v_plt.set_xticklabels(v_plt.get_xticklabels(),rotation = -0)
        #plt.show()

        #save plot out
        fig.savefig(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_pore_dist.png',bbox_inches='tight', dpi=300)
        
        #Close figure to hide previews
        plt.close(fig)

### Plotting porosity as function of pyridine as box plot

In [37]:
#Routing through each level of the dictionary initally parasing each of matching parameter sets
for matching in matched_para_comb:\
    #strip out whitespace from matching to find directories
    matchings = matching.replace(' ','') 
    #for each of the matching parameter sets route to the corresponding not matching parameters  
    for not_matching in matched_para_comb[matching]:
        #strip out whitespace from matching to find directories
        not_matchings = not_matching.replace(' ','') 
        #Make dataframe as holder for concatinated data
        con_df = pd.DataFrame()
        #make list of not_matching variables for ordering
        nm_order = []
        #make dataframe for summary data
        sum_df = pd.DataFrame()
        #make list for all the unique file identifiers
        uni_ids = []
        #file counter
        file_count = 0 
        #in each of these not matching parameters parase all of the listed files
        for filename in matched_para_comb[matching][not_matching]:
            #print(matching+not_matching)
            #print(filename)
            #create file to identify dataset
            file = filename[:-4]
            #print(file)
            #open distribution data as dataframe
            df = pd.read_csv(collated_data+filename, index_col = 0)
            #print(df.head())
            
            #extract not_matching parameter value
            #split between parameters
            nm = file.split(',')
            #split between parameter label and value
            nm = [i.split('-') for i in nm]
            #flatten list
            #nm = [item for sublist in nm for item in sublist]
            #print(nm)
            #exctract parameter value based on location within variables list
            #print(variables.index(not_matching))
            nm = nm[variables.index(not_matching)][1]
            #print(nm)
            
            #extend nm_order with not_matching parameter value
            nm_order.append(nm)
            
            #change data label to variable value
            df = df.rename(columns={'porosity':nm})
                                
            #Concatinate each recreated data set to eachother           
            con_df = pd.concat([con_df, df[nm]], axis=1, sort=False)
            #print(con_df)
            
            ##Calculating summary statistics from distriutions
            ##Calculating summary statistics from distriutions
            median = np.percentile(df[nm].dropna(), 50)
            IQR = iqr(df[nm].dropna())
            #in addition to the IQR calculate 25 and 75th quartiles
            quartile_25 = np.percentile(df[nm].dropna(), 25)
            quartile_75 = np.percentile(df[nm].dropna(), 75)
            mean = df[nm].mean()
            SD = df[nm].std()
            skew = df[nm].skew()
            #from calculation of skew determine whether to use median or mean
            if abs(skew) > 0.5:
                stat = 'median'
            else:
                stat = 'mean'

            #add each of the summary statstics and pair this with datasets
            sum_df.loc[file_count,'not_matching'] = nm
            sum_df.loc[file_count,'median'] = median
            sum_df.loc[file_count,'IQR'] = IQR
            sum_df.loc[file_count,'25_quartile'] = quartile_25
            sum_df.loc[file_count,'75_quartile'] = quartile_75
            sum_df.loc[file_count,'mean'] = mean
            sum_df.loc[file_count,'SD'] = SD
            sum_df.loc[file_count,'skew'] = skew
            sum_df.loc[file_count,'stat'] = stat
            #add column for not matching parameter
            sum_df.loc[file_count,not_matching] = nm
            #append unique IDs to list
            uni_ids.extend(df['unique_id'].dropna().tolist())
            
            #progress file count
            file_count = file_count +1 
        
        uni_ids = pd.DataFrame({'unique_id':uni_ids})
        #print(uni_ids)
        #print(con_df.head())
        sum_df = pd.concat([sum_df,uni_ids], axis=1)
        #Check summary dataframe
        #print(sum_df.head())
        #organise sum_df in terms of variable magniture
        sum_df.index = sum_df.index.astype(float)
        sum_df = sum_df.sort_index()
        #save summary dataframe out as CSV
        sum_df.to_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_summary'+'.csv')
        
        #convert multiple columns of data into two columns with header as variable and associated data
        data = pd.melt(con_df)
        #having converted dataframe want to remove any NANs
        data = data.dropna()

        #Find order of plots according to max value
        #ordered = data.groupby(['variable'])['value'].aggregate(np.median).reset_index().sort_values('value')
        #ordered_var = ordered['variable'].tolist()
        
        #Find order of plots from magnitude of nm variable, determined by ordering nm_order
        nm_order.sort(key = float, reverse=True)
        #print(nm_order)

        #create new template for figure
        fig, ax = plt.subplots()
        #plot box plot into figure
        b_plt = sns.boxplot( x='variable', y= 'value', data = data, order = nm_order, ax = ax) #order = ordered_var,
        
        #insert statistical annotations
        #before can add statistical annotation must create boxPairList from previous statistical comparison table
        #set which variable list controls order
        var_order = nm_order.copy()
        #create list for boxpairlist
        pre_boxPairList = []
        #for count of number of o values
        for index in range(len(var_order)):
            #to ensure that all combinations are considered again copy the uniquevalues
            avalues = var_order.copy() #colour hue
            #removing fixed variable so only consider changing variables
            avalues.remove(var_order[index])
            #considereing the appending value
            for index in range(len(avalues)):
                #let a = the file name and the ovalue which corresponds to the number within the list and pair them
                a = (avalues[index],var_order[index])
                #add the pair to the list of boxed pairs
                if avalues[index] != var_order[index]:
                    if a not in pre_boxPairList:
                        pre_boxPairList.append(a)
                else: pass
        #adding statistical annotation
        if len(pre_boxPairList) > 1:
            add_stat_annotation(x='variable', y= 'value', data = data, boxPairList=pre_boxPairList, 
                                test='Mann-Whitney', textFormat='star', loc='inside', verbose = 0,order = nm_order, ax = ax) #order = ordered_var,
        
        
        #Set plot labels
        #First add label to the x-axis to describe the variable considered
        #Retreve xlabel axis associated with variable parameter
        xlabel = variable_label[not_matching]

        #Add correct x tick labels
        #initially retreve the existing key labels
        labels = [t.get_text()  for t in ax.get_xticklabels()]
        #print(labels)

        xlabel_list = []
        for label in labels:
            #print(label)
            #For polymer solution crossreference variable with name
            if 'solution_name' in not_matching:
                #initilise dictionary of polymer solution names
                polysolkey = {'0.0':'Trial','1.0':'Initial' ,'2.0':'S1','3.0':'S2','4.0':'S3','5.0':'S4','6.0':'S5'}
                #change label to update polymer solution name
                label = polysolkey[label]

            #append list to list of variable labels
            xlabel_list.append(label)
        
        
        #print(xlabel_list)
        #using list of variable labels update x-axis tick labels
        b_plt.set_xticklabels(xlabel_list)
        #set the x and y axis labels
        b_plt.set(xlabel=xlabel, ylabel='Porosity (%)') #
        #set the xlabels rotation
        #b_plt.set_xticklabels(v_plt.get_xticklabels(),rotation = -0)
        #plt.show()

        #save plot out
        fig.savefig(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity.png',bbox_inches='tight', dpi=300)
                
        #Close figure to hide previews
        plt.close(fig)

## Porosity as a function of pyridine concentration

### fitting functions

In [38]:
def expfit(xdata,ydata,yerr,bounds):
    from common_fits import LogFunc
    import numpy as np
    ####Fit log curve with all scalars#####
    #using Scipy.optimise.curvefit then fit log function to the data to extract the curve parameters
    #setting bounds to prevent -ve log values
    bounds = bounds
    #fit the curve
    popt, pcov = curve_fit(ExpFunc,xdata, ydata,sigma=yerr,bounds=bounds)

    #each of the coefficients for the LogFunc to then be plotted
    a = popt[0]
    b = popt[1]
    c = popt[2]

    #calculate R^2 to get the goodness of fit
    residuals = ydata- ExpFunc(xdata, a,b,c)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    #print('log_fit'+str(r_squared))

    #having found the parameters then plot fitted curve onto figure
    min_x_val = xdata.min()
    max_x_val = xdata.max()
    expcurvex = np.linspace(min_x_val,max_x_val,1000)
    expcurvey = ExpFunc(expcurvex,a,b,c)
    
    return (m,b,c,r_squared,logcurvex,logcurvey)

def logfit(xdata,ydata,yerr,bounds):
    from common_fits import LogFunc
    import numpy as np
    ####Fit log curve with all scalars#####
    #using Scipy.optimise.curvefit then fit log function to the data to extract the curve parameters
    #setting bounds to prevent -ve log values
    bounds = bounds
    #fit the curve
    popt, pcov = curve_fit(LogFunc,xdata, ydata,sigma=yerr,bounds=bounds)

    #each of the coefficients for the LogFunc to then be plotted
    m = popt[0]
    b = popt[1]
    c = popt[2]

    #calculate R^2 to get the goodness of fit
    residuals = ydata- LogFunc(xdata, m,b,c)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    #print('log_fit'+str(r_squared))

    #having found the parameters then plot fitted curve onto figure
    min_x_val = xdata.min()
    max_x_val = xdata.max()
    logcurvex = np.linspace(min_x_val,max_x_val,1000)
    logcurvey = LogFunc(logcurvex,m,b,c)
    
    return (m,b,c,r_squared,logcurvex,logcurvey)


def linfit(xdata,ydata,yerr,bounds):
    from common_fits import linFunc
    import numpy as np
    ##Fit linier function
    #using Scipy.optimise.curvefit then fit lin function to the data to extract the curve parameters              
    popt, pcov = curve_fit(linFunc,xdata, ydata, sigma = yerr)

    #each of the coefficients for the LogFunc to then be plotted
    slope = popt[0]
    intercept = popt[1]

    #calculate R^2 to get the goodness of fit
    residuals = ydata- linFunc(xdata,slope,intercept)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    #print('lin_fit'+str(r_squared))

    #having found the parameters then plot fitted curve onto figure
    min_x_val = xdata.min()
    max_x_val = xdata.max()
    lincurvex = np.linspace(min_x_val,max_x_val,1000)
    lincurvey = linFunc(lincurvex,slope,intercept)
    
    return (slope,intercept,r_squared,lincurvex,lincurvey)

def tanhfit(xdata,ydata,yerr,bounds):
    popt, pcov = curve_fit(tanhFunc,xdata, ydata,sigma=yerr)

    #each of the coefficients for the LogFunc to then be plotted
    a = popt[0]
    b = popt[1]
    c = popt[2]
    d = popt[3]

    #calculate R^2 to get the goodness of fit
    residuals = ydata - tanhFunc(xdata, a,b,c,d)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    #print('tanh_fit'+str(r_squared))

    #having found the parameters then plot fitted curve onto figure
    min_x_val = xdata.min()
    max_x_val = xdata.max()
    tanhcurvex = np.linspace(min_x_val,max_x_val,1000)
    tanhcurvey = tanhFunc(tanhcurvex,a,b,c,d) 
    
    return (a,b,c,d,r_squared,tanhcurvex,tanhcurvey)    

In [42]:
#Routing through each level of the dictionary initally parasing each of matching parameter sets
for matching in matched_para_comb:
    #strip out whitespace from matching to find directories
    matchings = matching.replace(' ','') 
    #for each of the matching parameter sets route to the corresponding not matching parameters  
    for not_matching in matched_para_comb[matching]:
        #strip out whitespace from matching to find directories
        not_matchings = not_matching.replace(' ','') 
        #Make dataframe as holder for concatinated data
        con_df = pd.DataFrame()
        #make list of not_matching variables for ordering
        nm_order = []
        #make dataframe for summary data
        sum_df = pd.DataFrame()
        #in each of these not matching parameters parase all of the listed files
        for filename in matched_para_comb[matching][not_matching]:
            #print(matching+not_matching)
            #print(filename)
            #create file to identify dataset
            file = filename[:-4]
            #print(file)
            #open distribution data as dataframe
            df = pd.read_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_summary'+'.csv', index_col = 0)
            #print(df)
            
            
            #Using matpltlib to plot instead of seaborn to plot summary data
            #print(not_matching)
            if 'solution_name' in not_matching or 'flat_or_fibre' in not_matching :
                pass
            else:
                ###Plotting contineous summary data and fit curve if there are more than 3 data points
                if len(df[not_matchings]) > 3:
                    #Initilise subplots 
                    fig, ax = plt.subplots()
                    #print('pop')

                    #Using summary data table to extract data
                    x = df[not_matching].dropna()
                    xdata = np.asarray([1.0e-1 if x==0 else x for x in x])
                    ydata = np.asarray([1.0e-1 if x==0 else x for x in df['median'].dropna()])
                    yerr = ((df['median']-df['25_quartile']).dropna(),(df['75_quartile']-df['median']).dropna())
                   
                    ax.errorbar(xdata, ydata, yerr=yerr, fmt='o')
                    ax.set(xlabel=xlabel, ylabel='Porosity (%)') #
                    
                    #Set plot labels
                    #First add label to the x-axis to describe the variable considered
                    #Retreve xlabel axis associated with variable parameter
                    xlabel = variable_label[not_matching]

                    #set the x and y axis labels
                    ax.set(xlabel=xlabel, ylabel='Porosity (%)') #
                    #set the xlabels rotation
                    #ax.set_xticklabels(ax.get_xticklabels(),rotation = -0)
                    #plt.show()

                            ####Fit log curve with all scalars#####
                    yerr = np.asarray([1.0e-1 if x==0 else x for x in df['SD'].dropna()])
                    #using Scipy.optimise.curvefit then fit log function to the data to extract the curve parameters
                    #setting bounds to prevent -ve log values
                    bounds = (0,-np.inf, -np.inf), (np.inf, np.inf, np.inf)
                    #fit the curve
                    (m,b,c,r_squared,logcurvex,logcurvey) = logfit(xdata,ydata,yerr,bounds)
                    #plot
                    if r_squared > accept_r2:
                        ax.plot(logcurvex,logcurvey,'r', linewidth=1)
                    else:
                        pass
                    
                    #saving lot fit data into summary table
                    #get first row index key 
                    key = df.index[0]
                    #print(key)
                    df.loc[key,'m'] = m
                    df.loc[key,'b'] = b
                    df.loc[key,'c'] = c
                    df.loc[key,'log_r_squared'] = r_squared
                    #print(type(popt))

                    ##Fit linier function
                    #using Scipy.optimise.curvefit then fit lin function to the data to extract the curve parameters              
                    (m,c,r_squared,lincurvex,lincurvey) = linfit(xdata,ydata,yerr,bounds=None)
                    
                    #plot if R^2 is significant
                    if r_squared > accept_r2:
                        ax.plot(lincurvex,lincurvey,'b', linewidth=1)
                    else:
                        pass
                    
                    #saving lot fit data into summary table
                    #get first row index key 
                    df.loc[key,'slope'] = m
                    df.loc[key,'intercept'] = c
                    df.loc[key,'lin_r_squared'] = r_squared
                
                    #save plot out
                    fig.savefig(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_errorbarplt.png',bbox_inches='tight', dpi=300)
                    plt.close()
                    
                    #saving updated summary dataframe out
                    #print(df)
                    df.to_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_summary.csv')

## Porosity in terms of fibre diameter

### Fibre constants

In [17]:
def matchingdic(dat_loc,not_match,pass_lst):
    #import required libraries
    import os
    
    #create dictionary for matching parameters which do not contain the desired not matching parmeter
    matching_dic = {}
    #make secondary dictionary of keys as lists
    matching_dic_key = {}
    #itterating through each of the directories in the processed fibre directory
    for directory in os.listdir(dat_loc):
        #making sure to skip each of the unwanted directories in pass list
        if directory in pass_lst:
            pass
        else:
            #convert directory to string to list 
            directory = directory.split(',')
            #print(directory)
            #check if matching parameter list contains the desired non
            if not_match in directory:
                pass
            else:
                #convert list of matching parameters and assoicated values into separate lists of each            
                match_values = [x for x in directory[1::2]]
                #convert the list of matching variables into string to be used as a key in dictionary
                match_vars = [x for x in directory[::2]]
                match_varss = str(match_vars).strip('[]').replace("'",'').replace(' ','')
                #check if key is in matching_dic dictionary if not then add to dictionary along with associated matching parameter value
                
                if match_varss not in matching_dic.keys():
                    matching_dic.setdefault(match_varss, []).append(match_values)
                    if match_values not in matching_dic[match_varss]:
                        matching_dic.setdefault(match_varss, []).append(match_values)
                    else:              
                        pass
                
                
                #add list of parameters to dictionary of keys,matching_dic_key
                if match_varss not in matching_dic_key.keys():
                    matching_dic_key.setdefault(match_varss, []).append(match_vars)
                    if match_values not in matching_dic[match_varss]:
                        matching_dic_key.setdefault(match_varss, []).append(match_vars)
                    else:              
                        pass
                
    
    return (matching_dic,matching_dic_key)


#having created a dictionary of matching variables and associated parameter values 
#check metadata key to see if the meta data associated with the file being considered matches that of any in the dictionary for each key
#if find match break out of loop and go to directorary of data set with same matching parameters and not matching parameters
#open summary file and check r^2 values of each log and lin fit, extract parameters of which ever is highest 
#depending on which r^2 value is higher use equation and associated parameters to convert not matching parameter to new values
#plot new values for not matching variable to vs uncontrolled variable
#yay

In [2]:


#Having defined dictionary of matching parameter keys and associated parameter values for variable to be compared can then enter loop to compare them to 
#not matching parameter which is going to be converted. In this case it will be comparing micro-fibre diameter/pyridine with prosity/pyridine to get porosity/micro-fibre

#Routing through each level of the dictionary initally parasing each of matching parameter sets
for matching in matched_para_comb:
    #strip out whitespace from matching to find directories
    matchings = matching.replace(' ','') 
    #for each of the matching parameter sets route to the corresponding not matching parameters  
    for not_matching in matched_para_comb[matching]:
        #strip out whitespace from not_matching to find directories
        not_matchings = not_matching.replace(' ','') 
        
        #before starting conversion define dictionary of matching parameter keys and associated values to compare to that of files.
        #Do this outside for loop to minimise computations inside for loop

        #using matchingdic (as defined above) create dictionary of matching parameters used to separate data which does not contain not_matching param
        #dictionary has keys of matching parameters and their associated parameter values
        #location of all processed fibre data
        fibre_dat_loc = '/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/python_analysis/micro_fibre_dists/fibre_diameter/processed'
        #check which is the desired not matching variable
        not_match = not_matching
        #create a list of directories to skip this prevents further manipulations applied to matching directories from breaking
        pass_lst = ['raw_dist','hist_dist','.DS_Store']
        #start funciton returning matching_dic as dictionary of keys of each of the matching variables and associated parameter values
        matching_dic,matching_dic_key = matchingdic(fibre_dat_loc,not_match,pass_lst)
        #print(matching_dic)

        ##FINDING AND EXTRACTING APPROPRIATE CORRELATION AND PARAMETER VALUES
        #Only run through this loop once to extract parameter values 
        #use second file name loop to plot converted figures
        
        
        #only consider continious data so pass any datasets with catagorical uncontrolled variables
        if 'solution_name' in not_matching or 'flat_or_fibre' in not_matching :
            pass
        else:
            #create file count to minimise look ups
            file_count = 0
            
            #creating holder for which equation should be used to fit to data
            good_fit = None
            #create holders for parameters which for specified equation 
            m = 0
            b = 0
            c = 0
            #define a dictionary of pyridine concentrations and the associated micro-fibre diameter
            mfibre_pryd = {}
            
            #paraseing the [matching][not_matching] files 
            for filename in matched_para_comb[matching][not_matching]:
                
                if file_count == 0:
                
                    #open distribution data as dataframe
                    df = pd.read_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_summary.csv', index_col = 0)
                    #print(len(df[not_matchings].dropna()))


                    ###Plotting contineous summary data and fit curve if there are more than 3 data points
                    if len(df[not_matchings].dropna()) > 3:
                        #progress the file_count
                        file_count = file_count + 1

                        #cycling through matching_dic keys, retreve list of parameters associated with each
                        for key in matching_dic.keys():
                            parameters = matching_dic_key[key]
                            #reteve unique id to consider
                            unique_id = df['unique_id'].loc[0]
                            #print(unique_id)
                            #look up value for element in sublist and return all values as list 'parameter_values'
                            parameter_values = [str(sample_key.loc[sample_key['unique_id'] == unique_id, param].iloc[0]) 
                                                for sublist in parameters for param in sublist]
                            #convert keys into list of parameters to be used to recreate file name 
                            parameters = key.split(",")
                            #loop through parameters and values lists and recreate list name
                            match_name = []
                            for i in range(len(parameters)):
                                match_name.append(parameters[i])
                                match_name.append(parameter_values[i])
                            #strip combined list to recreate list
                            match_name = str(match_name).strip('[]').replace(' ','').replace("'",'')
                            #print(match_name)

                            #having recreate the file name the go to associated summary file
                            #setting the match file location
                            match_fibre_dat_loc = fibre_dat_loc+'/'+match_name+'/'+not_matchings
                            #then open the matched file location
                            matched_summary_df = pd.read_csv(match_fibre_dat_loc+'/'+match_name+'_'+not_matchings+'_summary.csv', index_col =0)
                            #reset index
                            matched_summary_df = matched_summary_df.reset_index()

                            #check if number of unique values in not_matching column is greater than 3 
                            #this ensures that the right key is being used in the conversion and esnsures enough conditions have been used to assess trend
                            uniquevalues = np.unique(matched_summary_df[not_matchings].values)
                            if len(uniquevalues) > 3:
                                #as long as enough points have been considered now need to check r^2 values to see if conversion is appropriate
                                #extract log_r_squared value
                                log_r_squared = matched_summary_df['log_r_squared'].loc[0]
                                #print(log_r_squared)
                                #extract lin_r_squared value
                                lin_r_squared = matched_summary_df['lin_r_squared'].loc[0]
                                if log_r_squared > lin_r_squared:
                                    #check if above accept_r2
                                    if log_r_squared > accept_r2:
                                        #if above extract parameter values 
                                        m = matched_summary_df['m'].loc[0]
                                        b = matched_summary_df['b'].loc[0]
                                        c = matched_summary_df['c'].loc[0]
                                        good_fit = 'log'
                                        break
                                    else:
                                        pass
                                else:
                                    if lin_r_squared > accept_r2:
                                        #if above extract parameter values 
                                        m = matched_summary_df['slope'].loc[0]
                                        b = matched_summary_df['intercept'].loc[0]
                                        good_fit = 'lin'
                                        break
                                    else:
                                        break
                    
                    #As opening summary data only need to do this once per not_matching!
                    
                    ####Plotting porosity vs fibre diameter with calculated values
                    #Having determined if there is a good fit associated with the controlled variable now want to convert
                    #extract existing data
                    
                    #retreve appropriate equation for conversion
                    if good_fit == 'log':
                        from common_fits import LogFunc as equation
                        parameters = (m,b,c)
                    elif good_fit == 'lin':
                        from common_fits import linFunc as equation
                        parameters = (m,b)
                    else:
                        pass
                    
                    if good_fit != None:
                        #convert extracted data with equation
                        x_values = df[not_matching].dropna()
                        x_values = np.asarray([1.0e-3 if x==0 else x for x in x_values])
                        converted = equation(x_values,*parameters)
                        y_values = df['median'].dropna()
                        yerr = ((df['median']-df['25_quartile']).dropna(),(df['75_quartile']-df['median']).dropna())
                        #xerr =
                        #print(converted)

                        #Add converted values to summary table
                        #df = pd.concat([df, pd.DataFrame({'converted_fibre_diameter':converted})], axis=1,sort=False)
                        #df['converted_fibre_diameter'] = converted
                        df.loc[:,'converted_fibre_diameter'] = pd.Series(converted)
                        #print(df.head())


                        #plot
                        #Initilise subplots 
                        fig, ax = plt.subplots()
                        ax.errorbar(converted, y_values, yerr, fmt='o')
                        ax.set(xlabel='Micro-fibre diameter ($\mu$m)', ylabel='Porosity (%)')



                        #fitting curve to points
                        xdata = converted
                        ydata = y_values
                        yerr = df['IQR'].dropna()
                        ####Fit log curve with all scalars#####
                        #setting bounds to prevent -ve log values
                        bounds = (0,-np.inf, -np.inf), (np.inf, np.inf, np.inf)
                        #fit the curve
                        (m,b,c,r_squared,logcurvex,logcurvey) = logfit(xdata,ydata,yerr,bounds)
                        #plot
                        #ax.plot(logcurvex,logcurvey,'r', linewidth=1)
                        if r_squared > accept_r2:
                            ax.plot(logcurvex,logcurvey,'r', linewidth=1)
                        else:
                            pass

                        #saving fit data into summary table
                        #get first row index key 
                        key = df.index[1]
                        #print(key)
                        df.loc[key,'m'] = m
                        df.loc[key,'b'] = b
                        df.loc[key,'c'] = c
                        df.loc[key,'log_r_squared'] = r_squared
                        #print(type(popt))

                        ####Fit exp curve with all scalars#####
                        #setting bounds to prevent -ve log values
                        bounds = (0,-np.inf, -np.inf), (np.inf, np.inf, np.inf)
                        #fit the curve
                        (a,b,c,r_squared,expcurvex,expcurvey) = logfit(xdata,ydata,yerr,bounds)
                        #print('exp_r_square_'+str(r_squared))
                        #plot
                        #ax.plot(logcurvex,logcurvey,'r', linewidth=1)
                        if r_squared > accept_r2:
                            ax.plot(expcurvex,expcurvey,'r', linewidth=1)
                        else:
                            pass

                        ####Fit lin curve with all scalars#####
                        #fit the curve
                        (m,c,r_squared,lincurvex,lincurvey) = linfit(xdata,ydata,yerr,bounds=None)
                        #plot
                        #ax.plot(lincurvex,lincurvey,'r', linewidth=1)
                        if r_squared > accept_r2:
                            ax.plot(lincurvex,lincurvey,'r', linewidth=1)
                        else:
                            pass

                        df.loc[key,'slope'] = m
                        df.loc[key,'intercept'] = c
                        df.loc[key,'lin_r_squared'] = r_squared


                        ####Fit tanh curve with all scalars#####
                        #fit the curve
                        (a,b,c,d,r_squared,tanhcurvex,tanhcurvey)= tanhfit(xdata,ydata,yerr,bounds)
                        #plot
                        #ax.plot(tanhcurvex,tanhcurvey,'r', linewidth=1)
                        if r_squared > 0.9:
                            ax.plot(tanhcurvex,tanhcurvey,'r', linewidth=1)
                        else:
                            pass

                        #save figure out
                        fig.savefig(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_calculated_microfibre_porosity_errorbarplt.png',bbox_inches='tight', dpi=300)
                        plt.close()

                        #saving updated summary dataframe out
                        #print(df)
                        df.to_csv(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_porosity_summary.csv')
                    
                    else:
                        ####Plotting porosity vs fibre diameter with observed values
                        #initilse figure
                        fig, ax = plt.subplots()
                        
                        #print(mfibre_pryd)
                        #itterate through the not matching values and convert them to fibre diameters with the dictionary of associated values                     
                        x_values = [matched_summary_df.loc[matched_summary_df[not_matching] == value, 'mean'].iloc[0] for value in df[not_matching].dropna()]
                        #print(x_values)
                        #Setting y_values for plot as media porosities
                        y_values = df['median'].dropna()
                        yerr = ((df['median']-df['25_quartile']).dropna(),(df['75_quartile']-df['median']).dropna())
                        
                        #retieveing error in x axis
                        #x_25_quart = [(value - matched_summary_df.loc[matched_summary_df['median'] == value, '25_quartile'].iloc[0]) for value in x_values]
                        #print(x_25_quart)
                        #x_75_quart = [(matched_summary_df.loc[matched_summary_df['median'] == value, '75_quartile'].iloc[0] - value) for value in x_values]
                        #print(x_75_quart)
                        #xerr = (x_25_quart,x_75_quart)
                        #xerr = [matched_summary_df.loc[matched_summary_df[not_matching] == value, 'SD'].iloc[0] for value in df[not_matching].dropna()]
                        
                        #xerr = ((matched_summary_df['median']-matched_summary_df['25_quartile']).dropna(),(matched_summary_df['75_quartile']-matched_summary_df['median']).dropna())
                        #xerr = matched_summary_df['median']
                        #print(matched_summary_df.head())
                        
                        #plotting converted values
                        ax.errorbar(x_values, y_values, yerr, fmt='o')
                        ax.set(xlabel='Micro-fibre diameter ($\mu$m)', ylabel='Porosity (%)')
                        
                        #Save figure out
                        fig.savefig(processed+'/'+matchings+'/'+not_matchings+'/'+matchings+'_'+not_matchings+'_observed_microfibre_porosity_errorbarplt.png',bbox_inches='tight', dpi=300)
                        plt.close()
                        
                #This level corresponds to regardless if file count is 0 or more
                else:
                    pass
               

NameError: name 'matched_para_comb' is not defined